# Format text from wikipedia biography

In [2]:
import pandas as pd
from _datetime import date

In [6]:
people_list = ['Glenn_Miller','Andrés_Segovia','Django_Reinhardt','Maria_Callas','Édith_Piaf','John_Lennon']
wikiId_List = ['64610','71932','9039','64966','64963','15852']
# wikiId = ['64610']

In [21]:
def readText(filename,folder):
    fileObject = open(folder+filename+'.txt', "r")
    data = fileObject.readlines()
    return data

def writeTextFile(filename,content):
    f = open('cleanText/'+filename+'.txt', "w")
    f.writelines(content)
    f.close()
    
def cleanText(text):
    ctext=[]
    for line in text:
        if "== See also ==" in line:
            break
        if not len(line.strip()) < 1 :
            ctext.append(line)
    return ctext

def identifySection(txt):
    '''
    args: text, the paragraph to identify
    return: section and subsection
    '''
    typeSection = 0
    if txt.startswith("==", 0, 2):
        typeSection = 1
    if txt.startswith("===", 0, 3):
        typeSection = 2
    
    return typeSection

In [14]:
df = pd.DataFrame()

for person in wikiId_list:
    # read text
    text = readText(person,"people/")
    # clean empty lines
    ctext= cleanText(text)
    print(len(ctext))
    writeTextFile(person,ctext)

47
66
97
218
111
92


In [42]:
# Structure: biography as a collection of paragraphs
## paragraphs as part of a section

for textFileName in wikiId_List:
    #dataframe to store all the information of the biography
    #read text
    text = readText(textFileName,"cleanText/")
    biography_df = pd.DataFrame({'paragraph':text})
    #index number
    biography_df['paragraphIndex'] = list(range(len(biography_df)))
    
    section_list = setSection(biography_df)
    #section name
    biography_df['section'] = section_list
    
    # save information
    biography_df.to_csv('cleanText/'+textFileName+'_codedParagraphs.csv',index=False)

In [41]:
# used to extract the sections and subsections
def setSection(biography_df):# row section
    sectionTitle_list = []
    lastTitle = ""
    sectionTitle = ""
    # for each paragraph in the dataframe
    for prg in biography_df.itertuples():
        #return the type of section: level 0, level 1 (belongs to previous section)
        typeSection = identifySection(prg.paragraph)
        if typeSection==0 and lastTitle == "":
            sectionTitle = "N/A"
        elif typeSection==1:
            lastTitle=prg.paragraph
            sectionTitle = prg.paragraph
        elif typeSection==2:
            sectionTitle = lastTitle+" \n "+prg.paragraph
        # add to a list
        sectionTitle_list.append(sectionTitle)
    return sectionTitle_list
    # row subsection